# Database from GPKG file

In this notebook, we load the data from the provided GPKG file and create a Noe4j database from it.


In [1]:
import geopandas as gpd
import pandas as pd
import fiona
# from ipyfilechooser import FileChooser
from geopandas import GeoSeries

import itertools
from operator import itemgetter

import numpy as np

from scipy.spatial import cKDTree

from shapely.geometry import Polygon, MultiPolygon, shape, Point, LineString


In [2]:
nt_df = gpd.read_file("data/NHN/NHN_02YL000_3_1_HN_NLFLOW_1.shp")

# convert to 2d

def ddd_to_dd_line(g):
    return LineString([xy[0:2] for xy in list(g.coords)]) 

nt_df['geometry'] = nt_df.geometry.apply(lambda x: ddd_to_dd_line(x))

## create segment id:

In [3]:
nt_df['row_index'] = nt_df.index
nt_df['row_index'] = nt_df['row_index'].astype('str')
nt_df['segment_id'] = nt_df['DATASETNAM'] + "-" + nt_df.row_index

In [4]:
nt_df.head()

,NID,VALDATE,ACQTECH,DATASETNAM,ACCURACY,PROVIDER,METACOVER,TYPE,DIRECTION,ISOLATED,...,GEONAMEDB,NAMEID_1,NAMEID_2,NAME_1,NAME_2,IDDATE,NAMEDATE,geometry,row_index,segment_id
0,fa6cd1faa5464cdc87d738eb23c32e79,None,5,02YL000,-1,2,-1,2,1,0,...,None,None,None,None,None,None,None,"LINESTRING (-57.20980 49.46456, -57.20972 49.4...",0,02YL000-0
1,5b0da0744a664e26a223b6fe357a688d,None,5,02YL000,-1,2,-1,2,1,0,...,None,None,None,None,None,None,None,"LINESTRING (-57.56010 49.53421, -57.56016 49.5...",1,02YL000-1
2,787c9b7f42374a42a0b8e55227b5b7d2,None,5,02YL000,-1,2,-1,2,1,0,...,None,None,None,None,None,None,None,"LINESTRING (-57.65649 48.97422, -57.65671 48.9...",2,02YL000-2
3,9f39f2dd252744489c609b0a9f0ce95f,None,5,02YL000,-1,2,-1,2,1,0,...,None,None,None,None,None,None,None,"LINESTRING (-57.65696 49.68104, -57.65617 49.6...",3,02YL000-3
4,4e20ab9dc67e4a64977a6cc8828aad99,None,5,02YL000,-1,2,-1,2,1,0,...,None,None,None,None,None,None,None,"LINESTRING (-57.54475 49.51510, -57.54469 49.5...",4,02YL000-4


In [5]:
nt_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 27444 entries, 0 to 27443
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   NID         27444 non-null  object  
 1   VALDATE     11855 non-null  object  
 2   ACQTECH     27444 non-null  int64   
 3   DATASETNAM  27444 non-null  object  
 4   ACCURACY    27444 non-null  int64   
 5   PROVIDER    27444 non-null  int64   
 6   METACOVER   27444 non-null  int64   
 7   TYPE        27444 non-null  int64   
 8   DIRECTION   27444 non-null  int64   
 9   ISOLATED    27444 non-null  int64   
 10  PRIORITY    27444 non-null  int64   
 11  FROM_JUNCT  27444 non-null  object  
 12  TO_JUNCT    27444 non-null  object  
 13  GEONAMEDB   2071 non-null   object  
 14  NAMEID_1    2071 non-null   object  
 15  NAMEID_2    0 non-null      object  
 16  NAME_1      2071 non-null   object  
 17  NAME_2      0 non-null      object  
 18  IDDATE      2071 non-null   object  
 

In [6]:
nt_df.NAME_1.unique()

array([None, 'Whites River', 'South Brook', 'East Adies River',
       'North Arm Brook', 'Hughes Brook', 'Rattler Brook', 'Steady Brook',
       'Lower Crabb Brook', 'Liverpool Brook', 'Beaver Brook',
       'Black Brook', 'Pynns Brook', 'Upper Humber River',
       'Barachois Brook', 'North Brook', 'Coal Brook', 'East Branch',
       'Rocky Brook', 'West Branch', 'Gillams Brook', 'Corner Brook',
       'Parsons Brook', 'Bakers Brook', 'Carp Creek',
       'Transmission Brook', 'Cooks Brook', 'Goose Arm Brook',
       'Adies River', 'Northeast Adies River', 'Wigwam Brook',
       'Old Mans Brook', 'Drill Hole Brook', 'Taylors Brook',
       'Goose Arm Brook North', 'Humber River', 'Gales Brook',
       'Crooked Feeder', 'Monkey Brook', "Cox's Brook", 'Nichols Brook',
       'Big Feeder Brook', 'Blow Me Down Brook', 'Little Falls Brook',
       'Stowbregs Brook', 'Little North Brook', 'Pratts Brook',
       'Deadwater Brook', 'Matthews Brook', 'Mine Brook',
       'Black Gulch Brook', 

In [7]:
nt_df['from_point'] = nt_df.apply(lambda x: x.geometry.coords[0], axis = 1)
nt_df['to_point'] = nt_df.apply(lambda x: x.geometry.coords[-1], axis = 1)
nt_df['number_of_points'] = nt_df.apply(lambda x: len(x.geometry.coords), axis = 1)


In [8]:
nt_df.head()

,NID,VALDATE,ACQTECH,DATASETNAM,ACCURACY,PROVIDER,METACOVER,TYPE,DIRECTION,ISOLATED,...,NAME_1,NAME_2,IDDATE,NAMEDATE,geometry,row_index,segment_id,from_point,to_point,number_of_points
0,fa6cd1faa5464cdc87d738eb23c32e79,None,5,02YL000,-1,2,-1,2,1,0,...,None,None,None,None,"LINESTRING (-57.20980 49.46456, -57.20972 49.4...",0,02YL000-0,"(-57.2097977, 49.464558)","(-57.2096062, 49.464952)",4
1,5b0da0744a664e26a223b6fe357a688d,None,5,02YL000,-1,2,-1,2,1,0,...,None,None,None,None,"LINESTRING (-57.56010 49.53421, -57.56016 49.5...",1,02YL000-1,"(-57.5601001, 49.5342086)","(-57.5605333, 49.5349059)",6
2,787c9b7f42374a42a0b8e55227b5b7d2,None,5,02YL000,-1,2,-1,2,1,0,...,None,None,None,None,"LINESTRING (-57.65649 48.97422, -57.65671 48.9...",2,02YL000-2,"(-57.6564862, 48.9742214)","(-57.6577213, 48.9743304)",7
3,9f39f2dd252744489c609b0a9f0ce95f,None,5,02YL000,-1,2,-1,2,1,0,...,None,None,None,None,"LINESTRING (-57.65696 49.68104, -57.65617 49.6...",3,02YL000-3,"(-57.6569614, 49.6810435)","(-57.6554065, 49.6811532)",3
4,4e20ab9dc67e4a64977a6cc8828aad99,None,5,02YL000,-1,2,-1,2,1,0,...,None,None,None,None,"LINESTRING (-57.54475 49.51510, -57.54469 49.5...",4,02YL000-4,"(-57.5447526, 49.5151011)","(-57.5445606, 49.5151585)",3


In [27]:
# will be used to index junctions
nt_df['segment_start_lon'] = nt_df.apply(lambda x: x.geometry.coords[0][0], axis = 1)
nt_df['segment_start_lat'] = nt_df.apply(lambda x: x.geometry.coords[0][1], axis = 1)
nt_df['segment_end_lon'] = nt_df.apply(lambda x: x.geometry.coords[-1][0], axis = 1)
nt_df['segment_end_lat'] = nt_df.apply(lambda x: x.geometry.coords[-1][1], axis = 1)


In [28]:
nt_df.head()

,NID,VALDATE,ACQTECH,DATASETNAM,ACCURACY,PROVIDER,METACOVER,TYPE,DIRECTION,ISOLATED,...,geometry,row_index,segment_id,from_point,to_point,number_of_points,segment_start_lon,segment_start_lat,segment_end_lon,segment_end_lat
0,fa6cd1faa5464cdc87d738eb23c32e79,None,5,02YL000,-1,2,-1,2,1,0,...,"LINESTRING (-57.20980 49.46456, -57.20972 49.4...",0,02YL000-0,"(-57.2097977, 49.464558)","(-57.2096062, 49.464952)",4,-57.209798,49.464558,-57.209606,49.464952
1,5b0da0744a664e26a223b6fe357a688d,None,5,02YL000,-1,2,-1,2,1,0,...,"LINESTRING (-57.56010 49.53421, -57.56016 49.5...",1,02YL000-1,"(-57.5601001, 49.5342086)","(-57.5605333, 49.5349059)",6,-57.560100,49.534209,-57.560533,49.534906
2,787c9b7f42374a42a0b8e55227b5b7d2,None,5,02YL000,-1,2,-1,2,1,0,...,"LINESTRING (-57.65649 48.97422, -57.65671 48.9...",2,02YL000-2,"(-57.6564862, 48.9742214)","(-57.6577213, 48.9743304)",7,-57.656486,48.974221,-57.657721,48.974330
3,9f39f2dd252744489c609b0a9f0ce95f,None,5,02YL000,-1,2,-1,2,1,0,...,"LINESTRING (-57.65696 49.68104, -57.65617 49.6...",3,02YL000-3,"(-57.6569614, 49.6810435)","(-57.6554065, 49.6811532)",3,-57.656961,49.681044,-57.655406,49.681153
4,4e20ab9dc67e4a64977a6cc8828aad99,None,5,02YL000,-1,2,-1,2,1,0,...,"LINESTRING (-57.54475 49.51510, -57.54469 49.5...",4,02YL000-4,"(-57.5447526, 49.5151011)","(-57.5445606, 49.5151585)",3,-57.544753,49.515101,-57.544561,49.515158


In [29]:
pt_df = pd.concat([nt_df['from_point'], nt_df['to_point']], axis = 0)
pt_df

0         (-57.2097977, 49.464558)
1        (-57.5601001, 49.5342086)
2        (-57.6564862, 48.9742214)
3        (-57.6569614, 49.6810435)
4        (-57.5447526, 49.5151011)
                   ...            
27439    (-57.3071663, 49.5121534)
27440     (-57.5807853, 49.633691)
27441    (-57.0179732, 49.5305027)
27442      (-57.09932, 49.4377062)
27443    (-56.9547163, 49.5767588)
Length: 54888, dtype: object

In [30]:
pt_df = pd.DataFrame(pt_df.unique(), columns=['junction'])
pt_df['junction_index'] = pt_df.index
pt_df['junction_index'] = pt_df['junction_index'].astype('str')
pt_df['junction_id'] = dataset_name + "-" + pt_df.junction_index
pt_df = pt_df.drop('junction_index', axis=1)

In [31]:
# pt_df.to_csv('junctions.csv', index =False)
nt_df.from_point[0]

(-57.2097977, 49.464558)

In [32]:
total_df = pd.merge(nt_df, pt_df, left_on='from_point', right_on='junction', how='left')
total_df = pd.merge(total_df, pt_df, left_on='to_point', right_on='junction', how='left', suffixes=('_from', '_to'))
total_df

,NID,VALDATE,ACQTECH,DATASETNAM,ACCURACY,PROVIDER,METACOVER,TYPE,DIRECTION,ISOLATED,...,to_point,number_of_points,segment_start_lon,segment_start_lat,segment_end_lon,segment_end_lat,junction_from,junction_id_from,junction_to,junction_id_to
0,fa6cd1faa5464cdc87d738eb23c32e79,None,5,02YL000,-1,2,-1,2,1,0,...,"(-57.2096062, 49.464952)",4,-57.209798,49.464558,-57.209606,49.464952,"(-57.2097977, 49.464558)",02YL000-0,"(-57.2096062, 49.464952)",02YL000-11485
1,5b0da0744a664e26a223b6fe357a688d,None,5,02YL000,-1,2,-1,2,1,0,...,"(-57.5605333, 49.5349059)",6,-57.560100,49.534209,-57.560533,49.534906,"(-57.5601001, 49.5342086)",02YL000-1,"(-57.5605333, 49.5349059)",02YL000-18267
2,787c9b7f42374a42a0b8e55227b5b7d2,None,5,02YL000,-1,2,-1,2,1,0,...,"(-57.6577213, 48.9743304)",7,-57.656486,48.974221,-57.657721,48.974330,"(-57.6564862, 48.9742214)",02YL000-2,"(-57.6577213, 48.9743304)",02YL000-26277
3,9f39f2dd252744489c609b0a9f0ce95f,None,5,02YL000,-1,2,-1,2,1,0,...,"(-57.6554065, 49.6811532)",3,-57.656961,49.681044,-57.655406,49.681153,"(-57.6569614, 49.6810435)",02YL000-3,"(-57.6554065, 49.6811532)",02YL000-7588
4,4e20ab9dc67e4a64977a6cc8828aad99,None,5,02YL000,-1,2,-1,2,1,0,...,"(-57.5445606, 49.5151585)",3,-57.544753,49.515101,-57.544561,49.515158,"(-57.5447526, 49.5151011)",02YL000-4,"(-57.5445606, 49.5151585)",02YL000-12117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27439,e17e93c9ca624bf18c714c55741aabb3,None,5,02YL000,-1,2,-1,2,1,0,...,"(-57.3071663, 49.5121534)",75,-57.312164,49.519072,-57.307166,49.512153,"(-57.3121636, 49.5190724)",02YL000-26387,"(-57.3071663, 49.5121534)",02YL000-26293
27440,06c586fbe9eb4a4c96939738177f2017,None,5,02YL000,-1,2,-1,2,1,0,...,"(-57.5807853, 49.633691)",2,-57.581129,49.633587,-57.580785,49.633691,"(-57.5811288, 49.6335875)",02YL000-26388,"(-57.5807853, 49.633691)",02YL000-26052
27441,cd0beb0ee6de4f1dbf22d79d21919f56,1986,5,02YL000,10,2,1,1,1,0,...,"(-57.0179732, 49.5305027)",41,-57.007558,49.537954,-57.017973,49.530503,"(-57.0075584, 49.5379543)",02YL000-26389,"(-57.0179732, 49.5305027)",02YL000-25808
27442,beb1b9db1d224b769c0467ef34bf4f05,None,5,02YL000,-1,2,-1,2,1,0,...,"(-57.09932, 49.4377062)",30,-57.098301,49.442427,-57.099320,49.437706,"(-57.0983015, 49.442427)",02YL000-26390,"(-57.09932, 49.4377062)",02YL000-25932


In [33]:
total_df.to_csv('y_segments_to_junction.csv')

# Segment to Segment relationship
Creating a relationship between segments that shows what segment is after or before a certain segment can be useful.

In [101]:
type_1 = pd.merge(nt_df[['segment_id', 'to_point']], nt_df[['segment_id', 'from_point']],\
         left_on='to_point', right_on='from_point', how='left', suffixes=('_first', '_second'))


In [83]:
nt_df.shape

(27444, 26)

In [102]:
type_2 = pd.merge(nt_df[['segment_id', 'from_point']], nt_df[['segment_id', 'to_point']],\
         left_on='from_point', right_on='to_point', how='left', suffixes=('_second', '_first')).dropna(axis =0).reset_index(drop=True)


In [105]:
seg_to_seg_df = pd.concat([type_1, type_2])

In [108]:
seg_to_seg_df[['segment_id_first', 'segment_id_second']].to_csv("data/segment_to_segment.csv",index=False)